# Module 3 Homework: Data Warehousing & BigQuery

## Question 1. Counting records

What is count of records for the 2024 Yellow Taxi Data?
- 65,623
- 840,402
- **20,332,093**
- 85,431,289

Answer:
```
SELECT count(*) FROM `hw3.tripdata`;

## Question 2. Data read estimation

Write a query to count the distinct number of PULocationIDs for the entire dataset on both the tables.
 
What is the **estimated amount** of data that will be read when this query is executed on the External Table and the Table?

- 18.82 MB for the External Table and 47.60 MB for the Materialized Table
- **0 MB for the External Table and 155.12 MB for the Materialized Table**
- 2.14 GB for the External Table and 0MB for the Materialized Table
- 0 MB for the External Table and 0MB for the Materialized Table

![02-query-table-external-estimate](02-query-table-external-estimate.png)

![02-query-table-nonpartition-estimate](02-query-table-nonpartition-estimate.png)

## Question 3. Understanding columnar storage

Write a query to retrieve the PULocationID from the table (not the external table) in BigQuery. Now write a query to retrieve the PULocationID and DOLocationID on the same table.

Why are the estimated number of Bytes different?
- **BigQuery is a columnar database, and it only scans the specific columns requested in the query. Querying two columns (PULocationID, DOLocationID) requires** 
reading more data than querying one column (PULocationID), leading to a higher estimated number of bytes processed.
- BigQuery duplicates data across multiple storage partitions, so selecting two columns instead of one requires scanning the table twice, 
doubling the estimated bytes processed.
- BigQuery automatically caches the first queried column, so adding a second column increases processing time but does not affect the estimated bytes scanned.
- When selecting multiple columns, BigQuery performs an implicit join operation between them, increasing the estimated bytes processed

## Question 4. Counting zero fare trips

How many records have a fare_amount of 0?
- 128,210
- 546,578
- 20,188,016
- **8,333**

![04-fareamount-zero](04-fareamount-zero.png)

## Question 5. Partitioning and clustering

What is the best strategy to make an optimized table in Big Query if your query will always filter based on tpep_dropoff_datetime and order the results by VendorID (Create a new table with this strategy)

- **Partition by tpep_dropoff_datetime and Cluster on VendorID**
- Cluster on by tpep_dropoff_datetime and Cluster on VendorID
- Cluster on tpep_dropoff_datetime Partition by VendorID
- Partition by tpep_dropoff_datetime and Partition by VendorID

```
CREATE OR REPLACE TABLE `hw3.partition_tripdata`
PARTITION BY DATE(tpep_dropoff_datetime)
CLUSTER BY VendorID AS (
  SELECT * FROM `hw3.nonpartition_tripdata`
);
```

## Question 6. Partition benefits


Use the materialized table you created earlier in your from clause and note the estimated bytes. Now change the table in the from clause to the partitioned table you created for question 5 and note the estimated bytes processed. What are these values? 


Choose the answer which most closely matches.
 

- 12.47 MB for non-partitioned table and 326.42 MB for the partitioned table
- **310.24 MB for non-partitioned table and 26.84 MB for the partitioned table**
- 5.87 MB for non-partitioned table and 0 MB for the partitioned table
- 310.31 MB for non-partitioned table and 285.64 MB for the partitioned table


partition:
select distinct(VendorID) from `hw3.partition_tripdata`
where tpep_dropoff_datetime >= '2024-03-01 00:00:00'
and tpep_dropoff_datetime < '2024-03-16 00:00:00'
```
This query will process 26.84 MB when run.
```

non partition:
select distinct(VendorID) from `hw3.nonpartition_tripdata`
where tpep_dropoff_datetime >= '2024-03-01 00:00:00'
and tpep_dropoff_datetime < '2024-03-16 00:00:00'
```
This query will process 310.24 MB when run.
```

## Question 7. External table storage

Where is the data stored in the External Table you created?

- Big Query
- Container Registry
- **GCP Bucket**
- Big Table

for external table, bigquery only store metadata, schema, location, format (csv, parquet etc).
when query, data still in bucket and bigquery process directly to them.

## Question 8. Clustering best practices

It is best practice in Big Query to always cluster your data:
- True
- **False**

Partitioning is usually essential; clustering is optional

## Question 9. Understanding table scans

No Points: Write a `SELECT count(*)` query FROM the materialized table you created. How many bytes does it estimate will be read? Why?


Because total record got from metadata, not physically do count. Table metadata have information  like total record etc.
